In [1]:
import os
import pandas as pd
import numpy as np
import time

In [2]:
Folder_path = "../Origin_Data/Result/"
File_Name = [
    "AIHUB_travel_DB.csv",
    "AIHUB_person_info_DB.csv",
    "AIHUB_faver_info_DB.csv",
    "AIHUB_visit_DB.csv"
]


def CSV_To_DataFrame(File_Path):
    df = pd.read_csv(File_Path, encoding="utf-8-sig")
    df = df.replace({np.NaN:None})
    return df

travel = CSV_To_DataFrame(Folder_path + File_Name[0])
person = CSV_To_DataFrame(Folder_path + File_Name[1])
faver = CSV_To_DataFrame(Folder_path + File_Name[2])
visit = CSV_To_DataFrame(Folder_path + File_Name[3])

C:\Users\MSI\AppData\Local\Temp\ipykernel_25524\3980280004.py:11: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(File_Path, encoding="utf-8-sig")


In [3]:
data_frame_list = [person, faver, travel, visit]

In [4]:
# 1. 여행 DB
travel_sql ="""
CREATE TABLE AI_HUB_travel_DB(
    TRAVEL_ID VARCHAR(255) PRIMARY KEY NOT NULL,
    TRAVEL_NM VARCHAR(255),
    TRAVELER_ID VARCHAR(255),
    TRAVEL_PURPOSE VARCHAR(255),
    TRAVEL_START_YMD DATE,
    TRAVEL_END_YMD DATE,
    MVMN_NM VARCHAR(255),
    TRAVEL_PERSONA VARCHAR(255)
)
# TRAVEL_ID TRAVEL_NM TRAVELER_ID TRAVEL_PURPOSE
# TRAVEL_START_YMD TRAVEL_END_YMD MVMN_NM TRAVEL_PERSONA
"""
travel_sql_input = """
INSERT INTO AI_HUB_travel_DB VALUES(%s, %s, %s, %s, %s, %s, %s, %s)
"""

# 2. 여행객 DB [사람정보]
pepole_info_sql = """
CREATE TABLE AI_HUB_person_info_DB(
    TRAVELER_ID VARCHAR(255) PRIMARY KEY NOT NULL,
    RESIDENCE_SGG_CD VARCHAR(255),
    GENDER VARCHAR(255),
    AGE_GRP int(10),
    EDU_NM VARCHAR(255),
    EDU_FNSH_SE VARCHAR(255),
    MARR_STTS VARCHAR(255),
    FAMILY_MEMB int(10),
    JOB_NM VARCHAR(255),
    JOB_ETC VARCHAR(255),
    INCOME VARCHAR(255),
    HOUSE_INCOME VARCHAR(255),
    TRAVEL_TERM VARCHAR(255),
    TRAVEL_NUM int(10),
    TRAVEL_COMPANIONS_NUM int(10),
    TRAVEL_STATUS_START_DAY DATE,
    TRAVEL_STATUS_END_DAY DATE
)
"""
# TRAVELER_ID RESIDENCE_SGG_CD GENDER AGE_GRP EDU_NM
# EDU_FNSH_SE MARR_STTS	FAMILY_MEMB	JOB_NM	JOB_ETC	
# INCOME HOUSE_INCOME TRAVEL_TERM TRAVEL_NUM TRAVEL_COMPANIONS_NUM
# TRAVEL_STATUS_START_DAY TRAVEL_STATUS_END_DAY
pepole_info_sql_input = """
INSERT INTO AI_HUB_person_info_DB VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 3. 여행객 DB [취향조사]
faver_info_sql = """
CREATE TABLE AI_HUB_faver_info_DB(
    TRAVELER_ID VARCHAR(255),
    TRAVEL_LIKE_SIDO_1 VARCHAR(255),
    TRAVEL_LIKE_SGG_1 VARCHAR(255),
    TRAVEL_LIKE_SIDO_2 VARCHAR(255),
    TRAVEL_LIKE_SGG_2 VARCHAR(255),
    TRAVEL_LIKE_SIDO_3 VARCHAR(255),
    TRAVEL_LIKE_SGG_3 VARCHAR(255),
    TRAVEL_STYL_1 int(10),
    TRAVEL_STYL_2 int(10),
    TRAVEL_STYL_3 int(10),
    TRAVEL_STYL_4 int(10),
    TRAVEL_STYL_5 int(10),
    TRAVEL_STYL_6 int(10),
    TRAVEL_STYL_7 int(10),
    TRAVEL_STYL_8 int(10),
    TRAVEL_STATUS_RESIDENCE VARCHAR(255),
    TRAVEL_STATUS_DESTINATION VARCHAR(255),
    TRAVEL_STATUS_ACCOMPANY VARCHAR(255),
    TRAVEL_MOTIVE_1 VARCHAR(255),
    foreign key (TRAVELER_ID) references AI_HUB_person_info_DB(TRAVELER_ID)
)
"""

faver_info_sql_input = """
INSERT INTO AI_HUB_faver_info_DB 
VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 5. 방문지 정보
visit_area_sql = """
CREATE TABLE AI_HUB_visit_area_DB(
    VISIT_AREA_ID VARCHAR(255),
    TRAVEL_ID VARCHAR(255),
    VISIT_ORDER int(10),
    VISIT_AREA_NM VARCHAR(255),
    VISIT_START_YMD DATE,
    VISIT_END_YMD DATE,
    ROAD_NM_ADDR VARCHAR(255),
    LOTNO_ADDR VARCHAR(255),
    X_COORD DECIMAL(10, 7),
    Y_COORD DECIMAL(10, 7),
    RESIDENCE_TIME_MIN DECIMAL(10,2),
    VISIT_AREA_TYPE_CD VARCHAR(255),
    REVISIT_YN VARCHAR(255),
    VISIT_CHC_REASON_CD VARCHAR(255),
    DGSTFN DECIMAL(10,2),
    REVISIT_INTENTION DECIMAL(10,2),
    RCMDTN_INTENTION DECIMAL(10,2)
)
"""
# VISIT_AREA_ID TRAVEL_ID VISIT_ORDER VISIT_AREA_NM VISIT_START_YMD	
# VISIT_END_YMD ROAD_NM_ADDR LOTNO_ADDR X_COORD Y_COORD	
# RESIDENCE_TIME_MIN VISIT_AREA_TYPE_CD REVISIT_YN VISIT_CHC_REASON_CD DGSTFN
# REVISIT_INTENTION RCMDTN_INTENTION
visit_area_sql_input = """
INSERT INTO AI_HUB_visit_area_DB 
VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [5]:
create_db_sql_dic = {
    "AI_HUB_person_info_DB":pepole_info_sql, 
    "AI_HUB_faver_info_DB":faver_info_sql, 
    "AI_HUB_travel_DB":travel_sql,
    "AI_HUB_visit_area_DB":visit_area_sql
}
insert_db_sql_dic = {
    "AI_HUB_person_info_DB":pepole_info_sql_input, 
    "AI_HUB_faver_info_DB":faver_info_sql_input,
    "AI_HUB_travel_DB":travel_sql_input, 
    "AI_HUB_visit_area_DB":visit_area_sql_input
}

In [6]:
import pymysql

#데이터 저장
user = 'root'
password = 'sql4869'
host='127.0.0.1' # loopback # localhost
port = 3306
database = 'sam_db' # DB 만들면 그거 이름

con = None;
cur = None;

try:
    con = pymysql.connect(host=host, user=user, password=password, port=port, db=database, charset='utf8')
    cur = con.cursor()
except Exception as e:
    print(e)
    print("DB 생성실패");

In [7]:
def CREATE_DATABASE(create_db_sql):
    try:
        cur.execute(create_db_sql)
    except Exception as e:
        print(e)
        print("생성실패")

def DELETE_DATABASE(delete_db_sql):
    try:
        cur.execute(delete_db_sql)
    except Exception as e:
        print(e)
        print("삭제실패")

def Insert_DATABASE(insert_db_sql, insert_data):
    try:
        for data in insert_data:
            try:
                cur.execute(insert_db_sql, data[1:])
            except Exception as e:
                print(data[1:])
                print(e)
                print("Insert 실패")
        con.commit();
    except:
        print("DB등록 실패")

for db_name, create_db_sql in create_db_sql_dic.items():
    check_db_sql = f"SHOW TABLES LIKE \'{db_name}\';"
    delete_db_sql = f"Delete from {db_name};"
    if not cur.execute(check_db_sql):
        CREATE_DATABASE(create_db_sql)
    else:
        DELETE_DATABASE(delete_db_sql)
    
for sql_dic, df in zip(insert_db_sql_dic.items(), data_frame_list):
    db_name, insert_db_sql = sql_dic
    check_db_sql = f"SHOW TABLES LIKE \'{db_name}\';"
    print(db_name)
    if not cur.execute(check_db_sql):
        create_db_sql = create_db_sql_dic.get(db_name)
        CREATE_DATABASE(create_db_sql)
    Insert_DATABASE(insert_db_sql, df.to_records().tolist())
    time.sleep(5)

cur.close();
con.close();

AI_HUB_person_info_DB
AI_HUB_faver_info_DB
AI_HUB_travel_DB
AI_HUB_visit_area_DB
(2210190011, 'd_d005999', 31, '서빈백사', '2022-10-19', '2022-10-19', None, '제주특별자치도 제주시 우도면 연평리', '126.9444344', '`', 30.0, '자연관광지', 'Y', '과거 경험이 좋아서', 5.0, 5.0, 5.0)
(1366, "Incorrect decimal value: '`' for column 'Y_COORD' at row 1")
Insert 실패
(2209030002, 'b_b002796', 8, '해맑은생선구이', '2022-09-03', '2022-09-03', None, None, '128.4207538', 'd34.8410505', 60.0, '식당/카페', 'N', '지인의 추천이 있어서', 4.0, 4.0, 4.0)
(1366, "Incorrect decimal value: 'd34.8410505' for column 'Y_COORD' at row 1")
Insert 실패
(2208300002, 'c_c000599', 2, '여수밥상갈치야', '2022-08-30', '2022-08-30', '전남 여수시 돌산읍 향일암로 378', None, '127.7874279!3', '34.6066052', 30.0, '식당/카페', 'N', '온라인(SNS, 블로그 등) 평가가 좋아서', 5.0, 5.0, 5.0)
(1366, "Incorrect decimal value: '127.7874279!3' for column 'X_COORD' at row 1")
Insert 실패
